In [1]:
from glob import glob

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from scipy.io import loadmat
from sklearn.metrics import classification_report, accuracy_score

import plotly.graph_objects as go

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def getFilesNew(basePath):
    dirs = glob(basePath+"/*/")
    #print(dirs)
    for _dir in dirs:
        files = glob(_dir+"*B?*.tiff")
        #print(files)
        yield files

In [5]:
S_sentinel_bands_forest = list(getFilesNew("/tmp/shared/2d_dataset/forest"))

In [6]:
S_sentinel_bands_town = list(getFilesNew("/tmp/shared/2d_dataset/town"))

In [7]:
S_sentinel_bands_water = list(getFilesNew("/tmp/shared/2d_dataset/water"))

In [8]:
S_sentinel_bands_field = list(getFilesNew("/tmp/shared/2d_dataset/field"))

In [9]:
S_sentinel_bands_sands = list(getFilesNew("/tmp/shared/2d_dataset/sands"))

In [10]:
def getFilesArrayNew(paths):
    l = []
    for i in paths:
        s = []
        for j in i:
            with rio.open(j, 'r') as f:
                s.append(f.read(1))
        l.append(s)
    arr_st_new = np.stack(l)
    return  arr_st_new

In [12]:
forest_st = getFilesArrayNew(S_sentinel_bands_forest);

In [13]:
town_st = getFilesArrayNew(S_sentinel_bands_town);

/usr/local/lib/python3.8/dist-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [14]:
water_st = getFilesArrayNew(S_sentinel_bands_water);

/usr/local/lib/python3.8/dist-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [15]:
field_st = getFilesArrayNew(S_sentinel_bands_field);

/usr/local/lib/python3.8/dist-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [16]:
sands_st = getFilesArrayNew(S_sentinel_bands_sands);

/usr/local/lib/python3.8/dist-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [18]:
town_st.shape

(55, 12, 100, 100)

In [19]:
t_st = np.moveaxis(town_st, 1, -1)
t_st.shape

(55, 100, 100, 12)

In [21]:
t_sst = t_st.reshape(-1, 20,20,12)

In [22]:
#Let's import some important Libraries
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.utils import np_utils

In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
from keras.layers import *
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

In [28]:
model_new = tf.keras.models.Sequential([

])

In [29]:
model_new.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy','MeanSquaredError'])

In [30]:
model_new.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [114]:
inputs = Input(shape=(20,20,12))
x = Conv2D(32, (3,3),strides=2, padding="same")(inputs)
x = BatchNormalization()(x)
x = Activation("relu")(x)

previous_block_activation = x

for filters in [32, 64, 128, 256]:
    x = Activation("relu")(x)
    x = SeparableConv2D(filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)

    x = Activation("relu")(x)
    x = SeparableConv2D(filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D(3, strides=2, padding="same")(x)

    residual = Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
    x = add([x, residual])
    previous_block_activation = x

for filters in [256, 128, 64]:
    x = Activation("relu")(x)
    x = Conv2DTranspose(filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)

    x = Activation("relu")(x)
    x = Conv2DTranspose(filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(2)(x)

    residual = UpSampling2D(2)(previous_block_activation)
    residual = Conv2D(filters, 1, padding="same")(residual)
    x = add([x, residual])  # Add back residual
    previous_block_activation = x

x = Activation("relu")(x)
x = Conv2DTranspose(32, (3,3), padding="same")(x)
x = BatchNormalization()(x)

x = Activation("relu")(x)
x = Conv2DTranspose(32, (3,3), padding="same")(x)
x = BatchNormalization()(x)

x = UpSampling2D(5)(x)
x = MaxPooling2D(4)(x)

residual = UpSampling2D(5)(previous_block_activation)
residual = MaxPooling2D(4)(residual)
residual = Conv2D(32, 1, padding="same")(residual)
x = add([x, residual])  # Add back residual

outputs = Conv2D(6, (3,3), activation="softmax", padding="same")(x)
#outputs = Reshape((10,10,6))(outputs)

In [115]:
previous_block_activation.shape

TensorShape([None, 8, 8, 64])

In [116]:
model = Model(inputs, outputs)

In [117]:
model.compile()

In [118]:
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 20, 20, 12)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_187 (Conv2D)            (None, 10, 10, 32)   3488        ['input_28[0][0]']               
                                                                                                  
 batch_normalization_322 (Batch  (None, 10, 10, 32)  128         ['conv2d_187[0][0]']             
 Normalization)                                                                                   
                                                                                           